In [1]:
%load_ext autoreload
%autoreload 2

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from hyperopt import hp, tpe, fmin, Trials
from tqdm import tqdm

import datetime
import copy
import json

import sys
sys.path.append('../../')

from data.dataloader import get_covid19india_api_data
from data.processing import get_data

from models.seir.seir_testing import SEIR_Testing
from models.seir.seirhd import SEIRHD
from models.seir.seir_movement import SEIR_Movement
from models.seir.seir_movement_testing import SEIR_Movement_Testing

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv
from utils.create_report import create_report
from utils.loss import Loss_Calculator
from viz import plot_forecast

## Load Covid19india Data

In [ ]:
dataframes = get_covid19india_api_data()

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
districts_to_show = [('Maharashtra', 'Mumbai')]

## Perform M1 and M2 fits

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(
        dataframes, state, district, train_period=7, val_period=7, model=SEIR_Testing,
        data_from_tracker=False, initialisation='intermediate', num_evals=700, 
        which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'],
        smooth_jump=True, smoothing_length=33, smoothing_method='weighted')
    predictions_dict[(state, district)]['m2'] = single_fitting_cycle(
        dataframes, state, district, train_period=7, val_period=0, num_evals=700,
        data_from_tracker=False, initialisation='intermediate', model=SEIR_Testing, 
        which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'],
        smooth_jump=True, smoothing_length=33, smoothing_method='weighted')
    
    predictions_dict[(state, district)]['state'] = state
    predictions_dict[(state, district)]['dist'] = district
    predictions_dict[(state, district)]['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict[(state, district)]['datasource'] = 'covid19api' if predictions_dict[(state, district)]['m1']['data_from_tracker'] else 'municipality'
    predictions_dict[(state, district)]['variable_param_ranges'] = predictions_dict[(state, district)]['m1']['variable_param_ranges']
    predictions_dict[(state, district)]['data_last_date'] = predictions_dict[(state, district)]['m2']['data_last_date']

## Create Master Loss Dataframe

### M1 Loss DataFrame

In [28]:
lc = Loss_Calculator()

In [30]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')

train                                    \
                   hospitalised total_infected deceased recovered   
Maharashtra Mumbai            1           0.49     3.55      3.68   

                            val                                    
                   hospitalised total_infected deceased recovered  
Maharashtra Mumbai        14.45           1.38    10.11        36

### M2 Loss DataFrame

In [31]:
lc.create_loss_dataframe_master(predictions_dict, 'm2')

train                                  
                   hospitalised total_infected deceased recovered
Maharashtra Mumbai         6.63           1.14     1.13     12.94

## Plot Forecasts

In [ ]:
for region in predictions_dict.keys():
    predictions_dict[region]['forecast'] = plot_forecast(predictions_dict[region], region, both_forecasts=True, error_bars=True)

## Create Report

In [ ]:
for region in predictions_dict.keys():
    create_report(predictions_dict[region])

## Create and Save Output CSV

In [ ]:
df_output = create_all_csvs(predictions_dict, icu_fraction=0.02)

In [ ]:
write_csv(df_output)